## Repo statistics for trydotnet

In [1]:
#r "nuget:Octokit, 0.32.0"
#r "nuget:NodaTime, 2.4.6"
using Octokit;
using NodaTime;
using NodaTime.Extensions;
using XPlot.Plotly;

In [3]:
var today = SystemClock.Instance.InUtc().GetCurrentDate();
var startOfTheMonth = today.With(DateAdjusters.StartOfMonth);
var startOfTheYear = new LocalDate(today.Year, 1, 1).AtMidnight();

In [4]:
var options = new ApiOptions();
var gitHubClient = new GitHubClient(new ProductHeaderValue("notebook"));

In [5]:
var createdIssuesRequest = new RepositoryIssueRequest
            {
                Since = startOfTheMonth.ToDateTimeUnspecified(),
                Filter = IssueFilter.Created
            };
var closedIssuesRequest = new RepositoryIssueRequest
            {
                Since = startOfTheMonth.ToDateTimeUnspecified(),
                State = ItemStateFilter.Closed
            };
var thisYearIssuesRequest = new RepositoryIssueRequest
            {
                Since = startOfTheYear.ToDateTimeUnspecified()
            };

In [6]:
var createdThisMonth = await gitHubClient.Issue.GetAllForRepository("dotnet", "try", createdIssuesRequest);
var closedThisMonth = await gitHubClient.Issue.GetAllForRepository("dotnet", "try", closedIssuesRequest);
var thisYearIssues = await gitHubClient.Issue.GetAllForRepository("dotnet", "try", thisYearIssuesRequest);

Unhandled Exception: (1,86): error CS1503: Argument 3: cannot convert from 'Octokit.RepositoryIssueRequest' to 'Octokit.ApiOptions'
(2,85): error CS1503: Argument 3: cannot convert from 'Octokit.RepositoryIssueRequest' to 'Octokit.ApiOptions'
(3,84): error CS1503: Argument 3: cannot convert from 'Octokit.RepositoryIssueRequest' to 'Octokit.ApiOptions'

(1,86): error CS1503: Argument 3: cannot convert from 'Octokit.RepositoryIssueRequest' to 'Octokit.ApiOptions'
(2,85): error CS1503: Argument 3: cannot convert from 'Octokit.RepositoryIssueRequest' to 'Octokit.ApiOptions'
(3,84): error CS1503: Argument 3: cannot convert from 'Octokit.RepositoryIssueRequest' to 'Octokit.ApiOptions'

In [7]:
var openSoFar = createdThisMonth.Where(i => i.State.StringValue == "open");
var openByMonth = openSoFar.GroupBy(i => new { i.CreatedAt.Year, i.CreatedAt.Month})
                .Select(g => new {Date = g.Key, Count = g.Count()});
var closedSoFar = thisYearIssues.Where(i => i.State.StringValue == "closed");
var closedByMonth = closedSoFar.GroupBy(i => new { i.ClosedAt.Value.Year, i.ClosedAt.Value.Month})
                .Select(g => new {Date = g.Key, Count = g.Count()});

Unhandled Exception: (1,17): error CS0103: The name 'createdThisMonth' does not exist in the current context
(4,19): error CS0103: The name 'thisYearIssues' does not exist in the current context

(1,17): error CS0103: The name 'createdThisMonth' does not exist in the current context
(4,19): error CS0103: The name 'thisYearIssues' does not exist in the current context

In [8]:
//display(createdThisMonth.Select(i => new {i.CreatedAt, i.Title, State = i.State.StringValue,  i.Number}));
var lineChart = Chart.Line(createdThisMonth.GroupBy(i => new DateTime(i.CreatedAt.Year,i.CreatedAt.Month, i.CreatedAt.Day)).Select(g => new Tuple<DateTime,int>(g.Key, g.Count())));
lineChart.WithTitle("Issue Open By Day");
display(lineChart);

Unhandled Exception: (2,28): error CS0103: The name 'createdThisMonth' does not exist in the current context

(2,28): error CS0103: The name 'createdThisMonth' does not exist in the current context

In [9]:
display(openSoFar.Select(i => new {i.CreatedAt, i.Title, State = i.State.StringValue,  i.Number}));

Unhandled Exception: (1,9): error CS0103: The name 'openSoFar' does not exist in the current context

(1,9): error CS0103: The name 'openSoFar' does not exist in the current context

In [10]:
//display(openByMonth);
var lineChart = Chart.Line(openByMonth.Select(g => new Tuple<DateTime, int>(new DateTime(g.Date.Year, g.Date.Month, 1),g.Count)));
lineChart.WithTitle("Issue Open By Month");
display(lineChart);

Unhandled Exception: (2,28): error CS0103: The name 'openByMonth' does not exist in the current context
(2,120): error CS1503: Argument 2: cannot convert from 'method group' to 'int'

(2,28): error CS0103: The name 'openByMonth' does not exist in the current context
(2,120): error CS1503: Argument 2: cannot convert from 'method group' to 'int'